# Spotify Cold Start Models
Seth Billiau, William Drew, Sarah Lucioni


## k-NN and k-Means Clustering Models

Our first set of models are k-Nearest Neighbor based models. These models exhibit a more tailored playlist. Our next set of models are k-Means Clustering based models. These exhibit a more computer selected playlist. By creating both sets of models, we will be able to test "human selection" vs. "computer selection."

We will evaluate each song by it's audio features determined via the Spotify API. The audio features we will use are:

- acousticness
- danceability
- energy
- instrumentalness
- liveness
- speechiness
- tempo

Spotify normalizes all values except tempo. 

The model variations we will explore include: 
- (**distance metric**) euclidean distance vs. cosine similarity
- (**tempo variations**) raw tempo vs. not using tempo  

In [1]:
#RUN THIS CELL 
import requests
from IPython.core.display import HTML
styles = requests.get("https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/cs109.css").text
HTML(styles)

In [2]:
import numpy as np
import pandas as pd
from scipy.stats import ttest_ind
pd.set_option('display.max_columns', 25)

from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.cluster import KMeans
import random

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set()

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

import pickle

First, we need to initiate access to the Spotify API. We need to set our API keys and set up authentication with the API.

In [3]:
# https://stackoverflow.com/questions/30557409/spotify-api-post-call-response-415
# https://stackoverflow.com/questions/30557409/spotify-api-post-call-response-415
def initiate_api():    
    client_id = "9cd3dd2ea2cf492ca28ab0247a79d781"
    client_secret = "11c972ad002843e9be5ecc31f022dd6e"
    grant_type = 'client_credentials'
    body_params = {'grant_type' : grant_type}
    url = 'https://accounts.spotify.com/api/token'
    response = requests.post(url, data=body_params, auth = (client_id, client_secret)) 
    client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
    return spotipy.Spotify(client_credentials_manager=client_credentials_manager)

sp = initiate_api()

Let's now load the song data that we will use to train our models.

In [4]:
songs_df = pd.read_pickle("william/pickles/songs_30k_dropped.pkl")
songs_df.head()

,index,acousticness,analysis_url,danceability,duration_ms,energy,id,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,track_href,type,uri,valence
53,1,0.000032,https://api.spotify.com/v1/audio-analysis/1MYY...,0.343,70294,0.975,1MYYt7h6amcrauCOoso3Gx,0.991,1,0.0515,-2.502,0,0.1350,119.883,4,https://api.spotify.com/v1/tracks/1MYYt7h6amcr...,audio_features,spotify:track:1MYYt7h6amcrauCOoso3Gx,0.133
54,2,0.000023,https://api.spotify.com/v1/audio-analysis/3x2m...,0.414,65306,0.959,3x2mJ2bjCIU70NrH49CtYR,0.933,7,0.3680,-4.299,0,0.0378,145.911,4,https://api.spotify.com/v1/tracks/3x2mJ2bjCIU7...,audio_features,spotify:track:3x2mJ2bjCIU70NrH49CtYR,0.589
55,3,0.053400,https://api.spotify.com/v1/audio-analysis/1Pm3...,0.522,108532,0.205,1Pm3fq1SC6lUlNVBGZi3Em,0.235,2,0.0985,-7.986,1,0.0376,103.868,4,https://api.spotify.com/v1/tracks/1Pm3fq1SC6lU...,audio_features,spotify:track:1Pm3fq1SC6lUlNVBGZi3Em,0.353
59,7,0.201000,https://api.spotify.com/v1/audio-analysis/7dkb...,0.493,226000,0.969,7dkbEHIMLoeuG4zXGmzhEH,0.655,2,0.2470,-3.282,0,0.0463,170.581,4,https://api.spotify.com/v1/tracks/7dkbEHIMLoeu...,audio_features,spotify:track:7dkbEHIMLoeuG4zXGmzhEH,0.340
88,36,0.007740,https://api.spotify.com/v1/audio-analysis/0hBb...,0.293,658987,0.787,0hBby0yygBY1u3m6tSpZgC,0.496,5,0.3920,-8.841,1,0.0438,165.307,4,https://api.spotify.com/v1/tracks/0hBby0yygBY1...,audio_features,spotify:track:0hBby0yygBY1u3m6tSpZgC,0.138


In [5]:
cols = ["acousticness", "danceability", "energy", "instrumentalness", "liveness", "speechiness", "tempo"]
cols_no_tempo = ["acousticness", "danceability", "energy", "instrumentalness", "liveness", "speechiness"]
songs_df_w_features = songs_df[cols]
songs_df_no_tempo = songs_df[cols_no_tempo]

## k-NN Models

Our k-NN models will have two variants: metric and tempo. This leads to four k-NN models total. The four models are:

- k-NN using Euclidean distance and tempo
- k-NN using Euclidean distance and no tempo
- k-NN using Cosine Similarity and tempo
- k-NN using Cosine Similarity and no tempo

We decided to only look at raw tempo vs. no tempo values instead of also trying to normalize tempo. We came to this conclusion after testing normalized tempo and saw that the playlists typically consisted of songs with missing audio features. This created a lot of issues while trying to test, so we decided to omit normalization of tempo.

The **knn** function below is our general k-NN model. 

In [163]:
def knn(songs_df, seed_song, n, metric="euclid", tempo=False):
    '''
    inputs:
        songs_df : dataframe of the audio features of all songs 
        seed_song : the uri of the seed song 
        n : length of desired playlist
        metric : string, "dist" or "sim" for euclidean distance or cosine similarity as the distance metric
        tempo : whether or not tempo should be included
    outputs:
        dist : pandas Series with 
    '''
    cols = ["acousticness", "danceability", "energy", "instrumentalness", "liveness", "speechiness", "tempo"]
    cols_no_tempo = ["acousticness", "danceability", "energy", "instrumentalness", "liveness", "speechiness"]
    
    seed_song = pd.DataFrame(sp.audio_features(seed_song))

    # Calculate distance using the desired metric
    if metric == "sim":
        if tempo:
            dist = cosine_similarity(np.asarray(songs_df[cols]), np.asarray(seed_song[cols]))
        else:
            dist = cosine_similarity(np.asarray(songs_df[cols_no_tempo]), np.asarray(seed_song[cols_no_tempo]))
            
        dist = pd.Series(dist.reshape(-1,))
        topn_dist = dist.sort_values(ascending=False)[0:n]
    else:
        if tempo:
            dist = euclidean_distances(np.asarray(songs_df[cols]), np.asarray(seed_song[cols]))
        else:
            dist = euclidean_distances(np.asarray(songs_df[cols_no_tempo]), np.asarray(seed_song[cols_no_tempo]))
            
        dist = pd.Series(dist.reshape(-1,))
        topn_dist = dist.sort_values()[0:n]
    
    topn = topn_dist.index
    return topn

def playlist_printer(ids, songs_df, kmeans=False):
    '''
    print out the playlist from song indexes and return the audio features of the playlist
    
    inputs:
        ids : list of song indexes in df
        songs_df : dataframe containing songs
    output: 
        playlist_features : dataframe of the audio features of the playlist
    '''
    cols = ["acousticness", "danceability", "energy", "instrumentalness", "liveness", "speechiness", "tempo"]
    playlist_features = pd.DataFrame(columns = cols)
    
    for i in ids:
        if kmeans:
            track_uri = songs_df.loc[i].id
        else:
            track_uri = songs_df.iloc[i].id
        track = sp.track(track_uri)
        print(track['name'])
        print('by')
        for artist in track['artists']:
            print(artist['name'])
        print('-----------------------------------')
        
        playlist_features = playlist_features.append(pd.DataFrame(sp.audio_features(track_uri))[cols])
        
    return playlist_features

Below, we use the above **knn** function to define each of our four k-NN models.

In [175]:
def knn_euclid_tempo(songs_df, seed_song, n):
    topn_ids = knn(songs_df, seed_song, n, metric="euclid", tempo=True)
    pf = playlist_printer(topn_ids, songs_df)
    return pf.describe().loc["mean"]

def knn_euclid_no_tempo(songs_df, seed_song, n):
    topn_ids = knn(songs_df, seed_song, n, metric="euclid", tempo=False)
    pf = playlist_printer(topn_ids, songs_df)
    return pf.describe().loc["mean"]

def knn_sim_tempo(songs_df, seed_song, n):
    topn_ids = knn(songs_df, seed_song, n, metric="sim", tempo=True)
    pf = playlist_printer(topn_ids, songs_df)
    return pf.describe().loc["mean"]

def knn_sim_no_tempo(songs_df, seed_song, n):
    topn_ids = knn(songs_df, seed_song, n, metric="sim", tempo=False)
    pf = playlist_printer(topn_ids, songs_df)
    return pf.describe().loc["mean"]
    
def run_all_knn(songs_df, seed_song, n):
    print("k-NN, euclidean distance, tempo included")
    print("****************************************")
    mean_knn_euclid_tempo = knn_euclid_tempo(songs_df, seed_song, n)
    
    print("\nk-NN, euclidean distance, tempo NOT included")
    print("****************************************")
    mean_knn_euclid_no_tempo = knn_euclid_no_tempo(songs_df, seed_song, n)
    
    
    print("\nk-NN, cosine similarity, tempo included")
    print("****************************************")
    mean_knn_sim_tempo = knn_sim_tempo(songs_df, seed_song, n)
    
    print("\nk-NN, cosine similarity, tempo NOT included")
    print("****************************************")
    mean_knn_sim_no_tempo = knn_sim_no_tempo(songs_df, seed_song, n)
    
    means_df = pd.concat([mean_knn_euclid_tempo, mean_knn_euclid_no_tempo, 
                          mean_knn_sim_tempo, mean_knn_sim_no_tempo], 
                         axis=1)
    means_df = means_df.transpose()
    means_df = means_df.append(pd.DataFrame(sp.audio_features(seed_song))[["acousticness", "danceability", "energy", "instrumentalness", "liveness", "speechiness", "tempo"]])
    means_df.index = ["k-NN/euclid/tempo", "k-NN/euclid/no tempo", "k-NN/sim/tempo", "k-NN/sim/no tempo", "seed song"]
    means_df = means_df.fillna(0)
    return means_df

In [177]:
# Senorita = "0TK2YIli7K1leLovkQiNik"
# Yo-Yo Ma Cello Suite No. 1 in G Major = "61dYvvfIRtIDFuqZypPAta"
run_all_knn(songs_df, "0TK2YIli7K1leLovkQiNik", 5)

k-NN, euclidean distance, tempo included
****************************************
Beautiful Love - Radio Version
by
Angelika Dusk
Playmen
-----------------------------------
The Distance
by
San Cisco
-----------------------------------
Let's Reggae All Night
by
CSS
-----------------------------------
Rough Soul
by
GoldLink
April George
-----------------------------------
Stay Down
by
Mary J. Blige
-----------------------------------

k-NN, euclidean distance, tempo NOT included
****************************************
Like Or Like Like
by
Miniature Tigers
-----------------------------------
Nena
by
Miguel Bosé
-----------------------------------
Come Home
by
Antix
-----------------------------------
Turn It Around
by
neutral.
-----------------------------------
Tus Ojos
by
Belanova
-----------------------------------

k-NN, cosine similarity, tempo included
****************************************
Friends of Friends
by
Hospitality
-----------------------------------
Carla
by
Jamiroquai

,acousticness,danceability,energy,instrumentalness,liveness,speechiness,tempo
k-NN/euclid/tempo,0.016392,0.7592,0.5458,0.000854,0.10394,0.04388,116.9898
k-NN/euclid/no tempo,0.033120,0.7526,0.5406,0.002896,0.09112,0.03244,101.6630
k-NN/sim/tempo,0.034080,0.7550,0.5446,0.002437,0.09400,0.03510,116.8056
k-NN/sim/no tempo,0.039800,0.7718,0.5486,0.000318,0.09556,0.03172,109.6254
seed song,0.037000,0.7590,0.5400,0.000000,0.09450,0.02870,116.9470


## k-Means Clustering

k-Means Clustering is an unsupervised machine learning algorithm which aims to split *n* observations (songs) into *k* clusters based on the nearest mean. We will have two k-Means Clustering models: one using tempo, a second which omits tempo.

In [164]:
def kmeans(k, songs_df, seed_song, n, tempo=False):
    '''
    Given a seed song (track_uri), predict a playlist of n song indexes
    
    inputs:
        k : number of clusters
        songs_df : k-means clustering model
        seed_song : the uri of the seed song 
        n : number of songs in playlist
        tempo : whether or not tempo should be included
    outputs:
        playlist_ids : list of n song indexes in the dataframe which is the playlist
    '''
    if tempo:
        cols = ["acousticness", "danceability", "energy", "instrumentalness", "liveness", "speechiness", "tempo"]
        songs_af = songs_df[cols]
        kmeans = KMeans(k).fit(songs_af)
    else:
        cols = ["acousticness", "danceability", "energy", "instrumentalness", "liveness", "speechiness"]
        songs_af = songs_df[cols]
        kmeans = KMeans(k).fit(songs_af)
        
    songs_w_cluster = songs_af.copy()
    songs_w_cluster["cluster"] = kmeans.predict(songs_af)   
    
    seed_song = pd.DataFrame(sp.audio_features(seed_song))
    pred_cluster = kmeans.predict(seed_song[cols])[0]
    pred_cluster_songs = songs_w_cluster[songs_w_cluster == pred_cluster]
    
    playlist_ids = random.sample(list(pred_cluster_songs.index), n)
    
    return playlist_ids

In [184]:
def kmeans_tempo(k, songs_df, seed_song, n):
    topn_ids = kmeans(k, songs_df, seed_song, n, tempo=True)
    pf = playlist_printer(topn_ids, songs_df, True)
    return pf.describe().loc["mean"]

def kmeans_no_tempo(k, songs_df, seed_song, n):
    topn_ids = kmeans(k, songs_df, seed_song, n, tempo=False)
    pf = playlist_printer(topn_ids, songs_df, True)
    return pf.describe().loc["mean"]

def run_all_kmeans(k, songs_df, seed_song, n):
    print("k-Means, k = {}, tempo included".format(k))
    print("****************************************")
    mean_kmeans_tempo = kmeans_tempo(k, songs_df, seed_song, n)
    
    print("\nk-Means, k = {}, tempo NOT included".format(k))
    print("****************************************")
    mean_kmeans_no_tempo = kmeans_no_tempo(k, songs_df, seed_song, n)
    
    means_df = pd.concat([mean_kmeans_tempo, mean_kmeans_no_tempo], axis=1)
    means_df = means_df.transpose()
    means_df = means_df.append(pd.DataFrame(sp.audio_features(seed_song))[["acousticness", "danceability", "energy", "instrumentalness", "liveness", "speechiness", "tempo"]])
    means_df.index = ["k-means/k={}/tempo".format(k), "k-means/k={}/no tempo".format(k), "seed song"]
    means_df = means_df.fillna(0)
    return means_df

In [183]:
# Senorita = "0TK2YIli7K1leLovkQiNik"
# Yo-Yo Ma Cello Suite No. 1 in G Major = "61dYvvfIRtIDFuqZypPAta"
run_all_kmeans(50, songs_df, "0TK2YIli7K1leLovkQiNik", 5)

k-Means, k = 10, tempo included
****************************************
El Perdón - Noodles Remix
by
Nicky Jam
Enrique Iglesias
-----------------------------------
God Only Knows
by
MKTO
-----------------------------------
Who Do You Love
by
Tufts Beelzebubs
-----------------------------------
Glass Elevator
by
The Infamous Stringdusters
-----------------------------------
Pseudo
by
Inner Wave
-----------------------------------

k-Means, k = 10, tempo NOT included
****************************************
Lady
by
Little River Band
-----------------------------------
Kaleidoscope Heart
by
Sara Bareilles
-----------------------------------
In the Shadows
by
Story Of The Year
-----------------------------------
I Lost It
by
Lucinda Williams
-----------------------------------
Butterflies
by
Alana Lee
-----------------------------------


,acousticness,danceability,energy,instrumentalness,liveness,speechiness,tempo
k-means/k=10/tempo,0.258480,0.6354,0.70380,0.379000,0.11812,0.03644,113.576
k-means/k=10/no tempo,0.370033,0.5116,0.59212,0.000004,0.14768,0.08812,129.305
seed song,0.037000,0.7590,0.54000,0.000000,0.09450,0.02870,116.947
